#### Project: LeNet-5 classic network digit classifiers with MNIST datasets
Here I redesign the LeNet-5 architecture by replacing the pooling layers with Maxpooling and the hidden layers with relu instead of tanh and the output layers with softmax layers instead.

#### Packages selection
- The first things is to import all the neccesary packages needed for this project

In [2]:
import time
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image

# select GPU if cuda is available
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

#### Settings
- Configure the device
- define all the hyperparameters to be used and needed to be tuned to achive a better accuracy
- Load and explore the data

In [5]:
# device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# model architecture parameters
NUM_FEATURES = 32*32
NUM_CLASSES = 10

# others
GRAYSCALE = False

# dataset -> CIFAR-10
# Note: transforms.ToTensor() scale image from 0-1 range

train_dataset = datasets.CIFAR10(root='data',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)

test_dataset = datasets.CIFAR10(root='data',
                               train=False,
                               transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                         batch_size= BATCH_SIZE,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=False)

# check the dataset
for images, labels in train_loader:
    print("Image batch dimension", images.shape)
    print("Image label dimension", labels.shape)
    break

Files already downloaded and verified
Image batch dimension torch.Size([128, 3, 32, 32])
Image label dimension torch.Size([128])


#### Define the architecture of the model such as
- The number of input layers; which is determined by the features of the data
- Number of total hidden layers in the model consist of only Convolution layers
- The output layer node units is determined by the intended outcome to achieve

In [6]:
"""
Architecture
X -> Conv2d -> ReLU -> MaxPool -> Conv2d -> ReLU -> MaxPool -> Linear -> ReLU -> Linear -> ReLu -> Linear -> Softmax -> y
"""

class LeNet5(nn.Module):
    def __init__(self, NUM_CLASSES, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = NUM_CLASSES
        
        if grayscale:
            in_channels = 1
        else:
            in_channels = 3
        
        self.features = nn.Sequential(
            
            nn.Conv2d(in_channels, 6*in_channels, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6*in_channels, 16*in_channels, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5*in_channels, 120*in_channels),
            nn.ReLU(),
            nn.Linear(120*in_channels, 84*in_channels),
            nn.ReLU(),
            nn.Linear(84*in_channels, self.num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        outputs = self.classifier(x)
        probas = F.softmax(outputs, dim=1)
        return outputs, probas

#### Loss function and optimizer
- Instantiate the model
- define the specific Loss function to be used either cross entropy, MSELoss, etc
- define the optimization algorithm to be used either SGD, Adam, RMSprop, Momentum etc.

In [8]:
torch.manual_seed(RANDOM_SEED)
model = LeNet5(NUM_CLASSES, GRAYSCALE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

#### compute accuracy
- A function to compute train and test accuracy

In [9]:
def compute_accuracy(model, data_loader):
    correct_predictions, num_examples = 0, 0
    for features, labels in data_loader:
        features = features.to(device)
        labels = labels.to(device)
        output, probas = model(features)
        _,predicted_labels = torch.max(probas, 1)
        num_examples += labels.size(0)
        correct_predictions += (predicted_labels == labels).sum()
    return correct_predictions.float() / num_examples *100
    

#### Training a model requires the following steps
- Reset all the gradients to zero (0)
- Make a forward pass (make a prediction)
- Calculate the loss
- Perform back propagation
- Update all the parameters (weight and biases)

In [10]:
start_time = time.time()
total_step = len(train_loader)
for epoch in range(NUM_EPOCHS):
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        
        # forward and back pass
        output, probas = model(features)
        loss = F.cross_entropy(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Logging
        if not i % 50:
            print('Epoch: %03d/%03d | Batch: %03d/%03d | Cost:%.4f'
                 %(epoch+1, NUM_EPOCHS, i, total_step, loss))
    model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' %(
    epoch+1, NUM_EPOCHS, compute_accuracy(model, train_loader)))
    print('Time Elapsed: %.2f min' %((time.time() - start_time) / 60 ))
print('Total Training Time: %.2f min' %((time.time() - start_time) / 60 ))

Epoch: 001/010 | Batch: 000/391 | Cost:2.3059
Epoch: 001/010 | Batch: 050/391 | Cost:2.0546
Epoch: 001/010 | Batch: 100/391 | Cost:1.8614
Epoch: 001/010 | Batch: 150/391 | Cost:1.7192
Epoch: 001/010 | Batch: 200/391 | Cost:1.5598
Epoch: 001/010 | Batch: 250/391 | Cost:1.4637
Epoch: 001/010 | Batch: 300/391 | Cost:1.4403
Epoch: 001/010 | Batch: 350/391 | Cost:1.4144
Epoch: 001/010 training accuracy: 47.29%
Time Elapsed: 1.61 min
Epoch: 002/010 | Batch: 000/391 | Cost:1.4807
Epoch: 002/010 | Batch: 050/391 | Cost:1.3707
Epoch: 002/010 | Batch: 100/391 | Cost:1.3756
Epoch: 002/010 | Batch: 150/391 | Cost:1.3793
Epoch: 002/010 | Batch: 200/391 | Cost:1.4210
Epoch: 002/010 | Batch: 250/391 | Cost:1.3806
Epoch: 002/010 | Batch: 300/391 | Cost:1.3048
Epoch: 002/010 | Batch: 350/391 | Cost:1.1524
Epoch: 002/010 training accuracy: 55.50%
Time Elapsed: 3.31 min
Epoch: 003/010 | Batch: 000/391 | Cost:1.2437
Epoch: 003/010 | Batch: 050/391 | Cost:1.2798
Epoch: 003/010 | Batch: 100/391 | Cost:1.304

#### Evaluation

In [11]:
with torch.set_grad_enabled(False):
    print('Test accuracy', compute_accuracy(model, test_loader))

Test accuracy tensor(68.2400)
